In [1]:
import pandas as pd
import numpy as np



### Get all the pillar names from the excel

In [2]:
ls ../..

README.md
UNDP Digital Assessment Data Framework Filename Matching V7.xlsx
dashboard/
data/
data_manifest.csv
data_manifest_instructions.md
process_manifest.js
process_raw_data.js
processed/
score/
sources/
undp-diagram.png


In [9]:
names = pd.read_excel('../../UNDP Digital Assessment Data Framework Filename Matching V7.xlsx')

In [10]:
col_names = ['Indicator','check', 'Data Source','Index','Filename']

In [11]:
names = names[col_names]

In [12]:
names.head()

Indicator           check  \
0                                          Countries             NaN   
1  Database of Global Administrative Areas (GADM,...             NaN   
2  High Resolution Population Density Maps + Demo...             NaN   
3  population density vs openstreetmap object den...             NaN   
4                                 Population Density  Infrastructure   

          Data Source  Index            Filename  
0      United Nations  False   list_of_countries  
1  GADM maps and data  False                 NaN  
2            Facebook  False                 NaN  
3              Kontur  False                 NaN  
4          World Bank  False  population_density

In [13]:
# get all the files per pillar
data_stats = names.groupby('check').agg({'Filename':'count','Indicator':'count'})

In [14]:
data_stats

Filename  Indicator
check                              
Business              20         25
Foundations            9         12
Government            12         15
Infrastructure        46         48
People                38         46
Regulation             6          7
Strategy               1          1

### Foundations

In [15]:
bnames = names[(names.check=='Foundations')&(~names.Filename.isna())]#&(names.Index==False)]

In [16]:
bnames.head(25)

Indicator        check  \
147                       Digital payments penetration  Foundations   
148  % of population with digital finance account -...  Foundations   
149  % of population with digital finance account -...  Foundations   
152  Identification For Development (ID4D) Global D...  Foundations   
153                            % of population with ID  Foundations   
154                 % of services that can be accessed  Foundations   
155                    can ID be used for transactions  Foundations   
156                            Is personal data siloed  Foundations   
157                                    Open data index  Foundations   

                   Data Source  Index                                 Filename  
147        Portulans Institute  TRUE              digital_payments_penetration  
148                 World Bank  False  %_population_digital_financial_services  
149                 World Bank  False  %_population_digital_financial_services  
152                World Bank   False                                 id4d_nid  
153                World Bank   False                                 id4d_nid  
154                World Bank   False                            id4d_services  
155                World Bank   False                            id4d_services  
156                 World Bank  False                            Egov_strategy  
157  Open Knowledge Foundation   True                            open_data_idx

In [17]:
# get list of names for all indicators
indicators = bnames.Indicator.unique()

In [18]:
# get all file names
bfiles = bnames.Filename.unique()

In [19]:
bfiles

array(['digital_payments_penetration',
       '%_population_digital_financial_services', 'id4d_nid',
       'id4d_services', 'Egov_strategy', 'open_data_idx'], dtype=object)

In [16]:
# ls digital-readiness-assessment-main/processed/

In [ ]:
##ict_goods and services not in process data

In [20]:
# formula for converting scale
def convert_rank(old_value, old_min=1, old_max=7, new_min=1, new_max=6 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

### 1. Digital payments penetration

In [21]:
indicators[0]

'Digital payments penetration'

In [22]:
# load data
indicator = indicators[0]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Digital payments penetration
digital_payments_penetration


In [23]:
df.head()

RANK COUNTRY/ECONOMY  VALUE   SCORE
0   1.0          Norway   0.85  100.00
1   2.0         Denmark   0.83   97.24
2   3.0         Finland   0.80   93.95
3   4.0          Sweden   0.80   93.08
4   5.0     Netherlands   0.76   89.01

In [24]:
# score looks like the one to use
df.describe()

RANK       VALUE       SCORE
count  122.000000  122.000000  122.000000
mean    61.500000    0.315738   36.076967
std     35.362409    0.209012   24.867406
min      1.000000    0.010000    0.000000
25%     31.250000    0.150000   16.235000
50%     61.500000    0.270000   30.870000
75%     91.750000    0.440000   51.110000
max    122.000000    0.850000  100.000000

In [43]:
# df.Indicator.unique()

In [27]:
# create standard columns
df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['SCORE'] 


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-100 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))



In [28]:
df

RANK Country Name  VALUE   SCORE  higher_is_better  \
0     1.0       Norway   0.85  100.00              True   
1     2.0      Denmark   0.83   97.24              True   
2     3.0      Finland   0.80   93.95              True   
3     4.0       Sweden   0.80   93.08              True   
4     5.0  Netherlands   0.76   89.01              True   
..    ...          ...    ...     ...               ...   
129   NaN      Jamaica    NaN     NaN              True   
130   NaN   Madagascar    NaN     NaN              True   
131   NaN         Oman    NaN     NaN              True   
132   NaN        Qatar    NaN     NaN              True   
133   NaN        Yemen    NaN     NaN              True   

                        Indicator  data_col  new_rank_score  
0    Digital payments penetration    100.00          6.0000  
1    Digital payments penetration     97.24          5.8620  
2    Digital payments penetration     93.95          5.6975  
3    Digital payments penetration     93.08          5.6540  
4    Digital payments penetration     89.01          5.4505  
..                            ...       ...             ...  
129  Digital payments penetration       NaN             NaN  
130  Digital payments penetration       NaN             NaN  
131  Digital payments penetration       NaN             NaN  
132  Digital payments penetration       NaN             NaN  
133  Digital payments penetration       NaN             NaN  

[134 rows x 8 columns]

In [31]:

# output scores to csv
df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].to_csv('../indicator_scores/foundations_{}_scores.csv'.format(bf), index=False)

In [34]:
df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].head(15)


Country Name                     Indicator  data_col  new_rank_score  \
0           Norway  Digital payments penetration    100.00          6.0000   
1          Denmark  Digital payments penetration     97.24          5.8620   
2          Finland  Digital payments penetration     93.95          5.6975   
3           Sweden  Digital payments penetration     93.08          5.6540   
4      Netherlands  Digital payments penetration     89.01          5.4505   
5      New Zealand  Digital payments penetration     86.73          5.3365   
6    United States  Digital payments penetration     84.40          5.2200   
7          Estonia  Digital payments penetration     82.82          5.1410   
8      Korea, Rep.  Digital payments penetration     82.64          5.1320   
9           Canada  Digital payments penetration     81.53          5.0765   
10       Australia  Digital payments penetration     79.37          4.9685   
11  Czech Republic  Digital payments penetration     75.65          4.7825   
12         Belgium  Digital payments penetration     73.15          4.6575   
13         Germany  Digital payments penetration     71.20          4.5600   
14          Poland  Digital payments penetration     70.34          4.5170   

    higher_is_better  
0               True  
1               True  
2               True  
3               True  
4               True  
5               True  
6               True  
7               True  
8               True  
9               True  
10              True  
11              True  
12              True  
13              True  
14              True

## 2. % of population with digital finance account - registered


In [35]:
indicator = indicators[1]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population with digital finance account - registered
%_population_digital_financial_services


In [108]:
# no DATE associated wiht the data

In [39]:
df.columns.tolist()

['Unnamed: 0',
 'Account (% age 15+)',
 'Account, male (% age 15+)',
 'Account, in labor force (% age 15+) ',
 'Account, out of labor force (% age 15+) ',
 'Account, female (% age 15+)',
 'Account, young adults (% ages 15-24)',
 'Account, older adults (% ages 25+)',
 'Account, primary education or less (% ages 15+) ',
 'Account, secondary education or more (% ages 15+) ',
 'Account, income, poorest 40% (% ages 15+)',
 'Account, income, richest 60% (% ages 15+) ',
 'Account, rural (% age 15+) ',
 'Financial institution account (% age 15+) ',
 'Financial institution account,male(% age 15+) ',
 'Financial institution account, in labor force(% age 15+) ',
 'Financial institution account, out of labor force (% age 15+) ',
 'Financial institution account,female(% age 15+) ',
 'Financial institution account,young adults(% age 15-24) ',
 'Financial institution account, older adults(% age 25+) ',
 'Financial institution account, primary education or less(% age 15+) ',
 'Financial institution ac

In [41]:
df[['data_country',
 'data_year','Unnamed: 0']].drop_duplicates()

data_country            data_year           Unnamed: 0
0            Low income           Low income           Low income
3   Lower middle income  Lower middle income  Lower middle income
5   Upper middle income  Upper middle income  Upper middle income
8                   NaN                  NaN                  NaN
11          High income          High income          High income

In [36]:
df.head(16)

Unnamed: 0 Account (% age 15+) Account, male (% age 15+)  \
0            Low income                  9%                       15%   
1            Low income                 10%                       16%   
2            Low income                 15%                       23%   
3   Lower middle income                 39%                       39%   
4   Lower middle income                 29%                       36%   
5   Upper middle income                 28%                       34%   
6   Upper middle income                 38%                       43%   
7   Upper middle income                 40%                       42%   
8                   NaN                 22%                       30%   
9                   NaN                 30%                       38%   
10                  NaN                 37%                       48%   
11          High income                 60%                       69%   
12          High income                 84%                       90%   
13          High income                 88%                       93%   
14  Upper middle income                 33%                       35%   
15  Upper middle income                 50%                       49%   

   Account, in labor force (% age 15+)   \
0                                   15%   
1                                   15%   
2                                   25%   
3                                   46%   
4                                   36%   
5                                   36%   
6                                   44%   
7                                   53%   
8                                   33%   
9                                   42%   
10                                  51%   
11                                  80%   
12                                  91%   
13                                  93%   
14                                  37%   
15                                  50%   

   Account, out of labor force (% age 15+)  Account, female (% age 15+)  \
0                                        2%                          3%   
1                                        4%                          4%   
2                                        4%                          7%   
3                                       31%                         39%   
4                                       12%                         22%   
5                                       15%                         23%   
6                                       29%                         34%   
7                                       27%                         38%   
8                                       11%                         14%   
9                                       18%                         22%   
10                                      24%                         26%   
11                                      24%                         47%   
12                                      49%                         68%   
13                                      57%                         76%   
14                                      28%                         32%   
15                                      50%                         51%   

   Account, young adults (% ages 15-24) Account, older adults (% ages 25+)  \
0                                    6%                                11%   
1                                    7%                                12%   
2                                   10%                                18%   
3                                   30%                                45%   
4                                   15%                                38%   
5                                   26%                                29%   
6                                   30%                                40%   
7                                   32%                                43%   
8                                   15%                      

In [97]:
# going to use the  score column since this is already an Index

In [102]:
# # convert rank into 1-6 - in order to get most recent data 
# df['data_col'] = df['Score'] 

# # going to assume index is between 1-100 but not 100% sure
# min_rank = 1 #df['data_col'].min()
# max_rank = 100 #df['data_col'].max()

# # transform 1-100 rank into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

# # need to invert score since higher rank is not better 
# # df['new_rank_score'] = (6-df['new_rank_score'])+1

In [107]:
# # prepare output
# df.rename(columns={'Country':'Country Name'}, inplace=True)
# df['Indicator'] = indicator
# df['higher_is_better'] = True

# df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].head(15)


# # output scores to csv
# df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].to_csv('../indicator_scores/{}_scores.csv'.format(bf), index=False)

## 3. % of population with digital finance account - active (90 days)

In [44]:
indicator = indicators[2]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population with digital finance account - active (90 days)
%_population_digital_financial_services


In [45]:
df.head()

Unnamed: 0 Account (% age 15+) Account, male (% age 15+)  \
0           Low income                  9%                       15%   
1           Low income                 10%                       16%   
2           Low income                 15%                       23%   
3  Lower middle income                 39%                       39%   
4  Lower middle income                 29%                       36%   

  Account, in labor force (% age 15+)   \
0                                  15%   
1                                  15%   
2                                  25%   
3                                  46%   
4                                  36%   

  Account, out of labor force (% age 15+)  Account, female (% age 15+)  \
0                                       2%                          3%   
1                                       4%                          4%   
2                                       4%                          7%   
3                                      31%                         39%   
4                                      12%                         22%   

  Account, young adults (% ages 15-24) Account, older adults (% ages 25+)  \
0                                   6%                                11%   
1                                   7%                                12%   
2                                  10%                                18%   
3                                  30%                                45%   
4                                  15%                                38%   

  Account, primary education or less (% ages 15+)   \
0                                               5%   
1                                               5%   
2                                               9%   
3                                              35%   
4                                              14%   

  Account, secondary education or more (% ages 15+)   ...  \
0                                                30%  ...   
1                                                23%  ...   
2                                                31%  ...   
3                                                44%  ...   
4                                                70%  ...   

  Mobile money account, female (% age 15+)   \
0                                       NaN   
1                                        0%   
2                                        1%   
3                                       NaN   
4                                       NaN   

  Mobile money account, young adults  (% age 15-24)   \
0                                                NaN   
1                                                 0%   
2                                                 0%   
3                                                NaN   
4                                                NaN   

  Mobile money account, older adults (% age 25+)   \
0                                             NaN   
1                                              0%   
2                                              1%   
3                                             NaN   
4                                             NaN   

  Mobile money account, primary education or less (% age 15+)   \
0                                                NaN             
1                                                 0%             
2                                                 0%             
3                                                NaN             
4                                                NaN             

  Mobile money account, secondary education or less (% age 15+)   \
0                                                NaN               
1                                                 0%               
2                                                 2%               
3                                                NaN               
4                                                N

No country or date data. Also not clear which column to use.

In [46]:
# dcol = 'Percentage of total trade in services'
# indicol = 'Category Label'
# cname = 'Economy Label'

# # filter most recent year and imports
# df = df[(df.Year==2019)&(df['Flow Label']=='Imports')]

# # create the standard columns
# df['higher_is_better'] = True
# df['Indicator'] =  df[indicol]+' '+ df['Flow Label']
# df['Country Name'] = df[cname]
# # df['2019 [YR2019]'] = df['2019 [YR2019]'].astype(float)
# df['data_col'] = df[dcol]


# # convert 1-100 %  into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
# df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
# df.to_csv('../indicator_scores/{}_scores.csv'.format(bf+'_imports'), index=False)

## 4. Identification For Development (ID4D) Global Dataset


In [47]:
indicator = indicators[3]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Identification For Development (ID4D) Global Dataset
id4d_nid


In [48]:
df.head()

Registered Population Above Cut-Off Age  Unnamed: 1 Country Characteristics  \
0                                      id         NaN                  Region   
1                                       1         NaN                     SAS   
2                                       2         NaN                     ECS   
3                                       3         NaN                     MEA   
4                                       4         NaN                     ECS   

             NID age Data Source Category  \
0  Mandatory NID Age                 Type   
1                  0                Voter   
2                 16               Direct   
3                 18                Voter   
4                  -                Voter   

                           Registered Population   By Gender By Rural/Urban  \
0  RPB (Registered Population Above Cut-off Age)    RPB Male      RPB Rural   
1                                     20,845,988  13,549,892            NaN   
2                                      4,455,772   2,267,673            NaN   
3                                     23,251,503         NaN            NaN   
4                                         24,512         NaN            NaN   

  By Age Group  data_country  data_year  
0     RPB 0-30           NaN        NaN  
1          NaN           NaN        NaN  
2    1,863,436           NaN        NaN  
3          NaN           NaN        NaN  
4          NaN           NaN        NaN

In [49]:
df.columns

Index(['Registered Population Above Cut-Off Age', 'Unnamed: 1',
       'Country Characteristics', 'NID age', 'Data Source Category',
       'Registered Population', 'By Gender', 'By Rural/Urban', 'By Age Group',
       'data_country', 'data_year'],
      dtype='object')

In [51]:
df[['data_country', 'data_year']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   data_country  0 non-null      float64
 1   data_year     0 non-null      float64
dtypes: float64(2)
memory usage: 3.2 KB


In [52]:
# dcol = 'Percentage of total trade in services'
# indicol = 'Category Label'
# cname = 'Economy Label'

# # filter most recent year
# df = df[(df.Year==2019)&(df['Flow Label']=='Exports')]

# # create standard columns
# df['higher_is_better'] = True
# df['Indicator'] =  df[indicol]+' '+ df['Flow Label']
# df['Country Name'] = df[cname]
# # df['2019 [YR2019]'] = df['2019 [YR2019]'].astype(float)
# df['data_col'] = df[dcol]


# # convert 1-100 %  into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
# df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
# df.to_csv('../indicator_scores/{}_scores.csv'.format(bf+'_exports'), index=False)

## % of population with ID


In [53]:
indicator = indicators[4]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population with ID
id4d_nid


In [55]:
df['Country Characteristics'].unique()

array(['Region', 'SAS', 'ECS', 'MEA', 'SSF', 'LCN', 'EAS', 'NAC'],
      dtype=object)

In [54]:
df.head()§

Registered Population Above Cut-Off Age  Unnamed: 1 Country Characteristics  \
0                                      id         NaN                  Region   
1                                       1         NaN                     SAS   
2                                       2         NaN                     ECS   
3                                       3         NaN                     MEA   
4                                       4         NaN                     ECS   

             NID age Data Source Category  \
0  Mandatory NID Age                 Type   
1                  0                Voter   
2                 16               Direct   
3                 18                Voter   
4                  -                Voter   

                           Registered Population   By Gender By Rural/Urban  \
0  RPB (Registered Population Above Cut-off Age)    RPB Male      RPB Rural   
1                                     20,845,988  13,549,892            NaN   
2                                      4,455,772   2,267,673            NaN   
3                                     23,251,503         NaN            NaN   
4                                         24,512         NaN            NaN   

  By Age Group  data_country  data_year  
0     RPB 0-30           NaN        NaN  
1          NaN           NaN        NaN  
2    1,863,436           NaN        NaN  
3          NaN           NaN        NaN  
4          NaN           NaN        NaN

In [56]:
# dcol = 'Percentage of total merchandise trade'
# indicol = 'IctProductCategory Label'
# cname = 'Economy Label'

# # filter most recent year and exports
# df = df[(df.Year==2019)&(df['Flow Label']=='Exports')]

# # create standard columns
# df['higher_is_better'] = True
# df['Indicator'] =  df[indicol]+' '+ df['Flow Label']
# df['Country Name'] = df[cname]
# # df['2019 [YR2019]'] = df['2019 [YR2019]'].astype(float)
# df['data_col'] = df[dcol]


# # convert 1-100 %  into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
# df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
# df.to_csv('../indicator_scores/{}_scores.csv'.format(bf+'_exports'), index=False)

## 6. % of services that can be accessed

In [57]:
indicator = indicators[5]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of services that can be accessed
id4d_services


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 11 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Legal Enablers to ID                   199 non-null    object
 1   Unnamed: 1                             202 non-null    object
 2   Country Characteristics                201 non-null    object
 3   Legal System Type                      203 non-null    object
 4   Foundational Civil Identification Law  203 non-null    object
 5   Data Protection Laws & Privacy Bills   203 non-null    object
 6   Freedom in the World (2018)            194 non-null    object
 7   Freedom on the Net (2017)              64 non-null     object
 8   Freedom in the Press (2017)            194 non-null    object
 9   data_country                           194 non-null    object
 10  data_year                              194 non-null    object
dtypes: object(11)
memor

In [60]:
df['Freedom on the Net (2017)'].unique()

array(['Status ', nan, 'PF', 'F', 'NF'], dtype=object)

In [58]:
df.head()

Legal Enablers to ID                          Unnamed: 1  \
0                    #  Economic Environment (0-30 points)   
1                    1                                  16   
2                    2                                  17   
3                    3                                  19   
4                    4                                   8   

  Country Characteristics              Legal System Type  \
0            Country Code              Legal System Type   
1                     AFG  civil + customary + religious   
2                     ALB                          civil   
3                     DZA              civil + religious   
4                     AND                          civil   

               Foundational Civil Identification Law  \
0                                     Key Act / Bill   
1  Law for Registration of Population Records, Ar...   
2    Law on ID Cards No. 8952, as amended, Arts. 1-3   
3  Décret No. 67/126 du 21/07/1967 Portant Instit...   
4                                                  -   

     Data Protection Laws & Privacy Bills Freedom in the World (2018)  \
0                          Key Act / Bill                      Status   
1                                    None                          NF   
2  Act on the Protection of Personal Data                          PF   
3                                    None                          NF   
4  Law on the protection of personal data                           F   

  Freedom on the Net (2017) Freedom in the Press (2017)  \
0                   Status         Press Freedom Status   
1                       NaN                          PF   
2                       NaN                          PF   
3                       NaN                          NF   
4                       NaN                           F   

                         data_country                           data_year  
0  Economic Environment (0-30 points)  Economic Environment (0-30 points)  
1                                  16                                  16  
2                                  17                                  17  
3                                  19                                  19  
4                                   8                                   8

In [61]:
# dcol = 'Percentage of total merchandise trade'
# indicol = 'IctProductCategory Label'
# cname = 'Economy Label'

# # filter most recent year and exports
# df = df[(df.Year==2019)&(df['Flow Label']=='Imports')]

# # create standard columns
# df['higher_is_better'] = True
# df['Indicator'] =  df[indicol]+' '+ df['Flow Label']
# df['Country Name'] = df[cname]
# # df['2019 [YR2019]'] = df['2019 [YR2019]'].astype(float)
# df['data_col'] = df[dcol]


# # convert 1-100 %  into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
# df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
# df.to_csv('../indicator_scores/{}_scores.csv'.format(bf+'_imports'), index=False)

## 7. can ID be used for transactions



In [62]:
indicator = indicators[6]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

can ID be used for transactions
id4d_services


In [63]:
df.head()

Legal Enablers to ID                          Unnamed: 1  \
0                    #  Economic Environment (0-30 points)   
1                    1                                  16   
2                    2                                  17   
3                    3                                  19   
4                    4                                   8   

  Country Characteristics              Legal System Type  \
0            Country Code              Legal System Type   
1                     AFG  civil + customary + religious   
2                     ALB                          civil   
3                     DZA              civil + religious   
4                     AND                          civil   

               Foundational Civil Identification Law  \
0                                     Key Act / Bill   
1  Law for Registration of Population Records, Ar...   
2    Law on ID Cards No. 8952, as amended, Arts. 1-3   
3  Décret No. 67/126 du 21/07/1967 Portant Instit...   
4                                                  -   

     Data Protection Laws & Privacy Bills Freedom in the World (2018)  \
0                          Key Act / Bill                      Status   
1                                    None                          NF   
2  Act on the Protection of Personal Data                          PF   
3                                    None                          NF   
4  Law on the protection of personal data                           F   

  Freedom on the Net (2017) Freedom in the Press (2017)  \
0                   Status         Press Freedom Status   
1                       NaN                          PF   
2                       NaN                          PF   
3                       NaN                          NF   
4                       NaN                           F   

                         data_country                           data_year  
0  Economic Environment (0-30 points)  Economic Environment (0-30 points)  
1                                  16                                  16  
2                                  17                                  17  
3                                  19                                  19  
4                                   8                                   8

In [66]:
# min_rank = 1
# max_rank = df['Country Name'].nunique()

# # transform 1-24 rank into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

# # need to invert score since higher rank is not better 
# # df['new_rank_score'] = (6-df['new_rank_score'])+1

In [67]:
# # prepare output
# df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
# df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
# df.to_csv('../indicator_scores/{}_scores.csv'.format(bf), index=False)

## 8. Is personal data siloed


In [68]:
indicator = indicators[7]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Is personal data siloed
Egov_strategy


In [69]:
df.head()

198 DG Systems & Services Unnamed: 2 Country Data  \
0  Group                     #      GTI-4        Level   
1      C                     1      -0.12          LIC   
2      A                     2       0.26         UMIC   
3      C                     3      -0.15         LMIC   
4      C                     4      -1.32          HIC   

                                  Digital Government UN e-Gov Dev Index  \
0                                       e-Government            eGov'20   
1                      https://mcit.gov.af/node/6938                169   
2                              https://e-albania.al/                 59   
3  https://www.mpttn.gov.dz/ar/content/%D8%A7%D9%...                120   
4                               http://www.govern.ad                 80   

              DG Institution DG Strategy Whole of Government GovTech  ...  \
0                 DG Org URL       DG St                 WoG      GT  ...   
1        http://mcit.gov.af/           2                   0       0  ...   
2       http://akshi.gov.al/           3                   1       1  ...   
3  https://www.mpttn.gov.dz/           2                   0       0  ...   
4      http://www.govern.ad/           3                   0       0  ...   

  Comparison of weights OECD DG 2019  \
0                   GTE        Score   
1                 0.442            -   
2                 0.748            -   
3                 0.281            -   
4                 0.336            -   

  GTMI > Correlation of indicators (without standardization)  \
0                                               Code           
1                                                AFG           
2                                                ALB           
3                                                DZA           
4                                                ADO           

  GTI > Factor analysis (without standardization) GTMI Raw Scores >   \
0                                            Code               Code   
1                                             AFG                AFG   
2                                             ALB                ALB   
3                                             DZA                DZA   
4                                             ADO                ADO   

  GTMI Standardized Z-Scores>   \
0                         Code   
1                          AFG   
2                          ALB   
3                          DZA   
4                          ADO   

  GTC > Correlation analysis with Standardized Scores  \
0                                               Code    
1                                                AFG    
2                                                ALB    
3                                                DZA    
4                                                ADO    

  GTF > Factor analysis with Standardized Scores data_country data_year  
0                                           Code        GTI-4     GTI-4  
1                                            AFG        -0.12     -0.12  
2                                            ALB         0.26      0.26  
3                                            DZA        -0.15     -0.15  
4                                            ADO        -1.32     -1.32  

[5 rows x 123 columns]

In [70]:
# # filter on relevant years
# df = df[(df['Time']==2017)&(df['Information and communication technologies']=='ICT-intensive')]

# # convert rank into 1-6 - in order to get most recent data 
# df['data_col'] = df['Value'] 

# # going to assume index is between 1-100 but not 100% sure
# min_rank = 0 #df['data_col'].min()
# max_rank = 100 #df['data_col'].max()

# # transform 1-147 rank into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

# # need to invert score since higher rank is not better 
# # df['new_rank_score'] = (6-df['new_rank_score'])+1

# # df_rank[['Country ISO3', 'Country Name','Indicator','data_col','new_rank_score']].head()

# df.rename(columns={'Country':'Country Name'}, inplace=True)
# df['Indicator'] = indicator
# df['higher_is_better'] = True

# df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].head(15)


# # output scores to csv
# df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].to_csv('../indicator_scores/{}_scores.csv'.format(bf), index=False)

## 9. Open data index

In [72]:
bnames

Indicator        check  \
147                       Digital payments penetration  Foundations   
148  % of population with digital finance account -...  Foundations   
149  % of population with digital finance account -...  Foundations   
152  Identification For Development (ID4D) Global D...  Foundations   
153                            % of population with ID  Foundations   
154                 % of services that can be accessed  Foundations   
155                    can ID be used for transactions  Foundations   
156                            Is personal data siloed  Foundations   
157                                    Open data index  Foundations   

                   Data Source  Index                                 Filename  
147        Portulans Institute  TRUE              digital_payments_penetration  
148                 World Bank  False  %_population_digital_financial_services  
149                 World Bank  False  %_population_digital_financial_services  
152                World Bank   False                                 id4d_nid  
153                World Bank   False                                 id4d_nid  
154                World Bank   False                            id4d_services  
155                World Bank   False                            id4d_services  
156                 World Bank  False                            Egov_strategy  
157  Open Knowledge Foundation   True                            open_data_idx

In [71]:
indicator = indicators[8]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Open data index
open_data_idx


FileNotFoundError: [Errno 2] No such file or directory: '../../processed/open_data_idx.csv'

### Score Aggregating

In [73]:
import os


In [77]:
# get list of files in scores folder
scores = os.listdir('../indicator_scores/')
scores = [s for s in scores if s.startswith('foundations')]

In [78]:
scores

['foundations_digital_payments_penetration_scores.csv']

In [79]:
# create a dataframe that concatenates all these file into one table
df = pd.concat([pd.read_csv('../indicator_scores/{}'.format(s)) for s in scores])    

In [80]:
df

Country Name                     Indicator  data_col  new_rank_score  \
0         Norway  Digital payments penetration    100.00          6.0000   
1        Denmark  Digital payments penetration     97.24          5.8620   
2        Finland  Digital payments penetration     93.95          5.6975   
3         Sweden  Digital payments penetration     93.08          5.6540   
4    Netherlands  Digital payments penetration     89.01          5.4505   
..           ...                           ...       ...             ...   
129      Jamaica  Digital payments penetration       NaN             NaN   
130   Madagascar  Digital payments penetration       NaN             NaN   
131         Oman  Digital payments penetration       NaN             NaN   
132        Qatar  Digital payments penetration       NaN             NaN   
133        Yemen  Digital payments penetration       NaN             NaN   

     higher_is_better  
0                True  
1                True  
2                True  
3                True  
4                True  
..                ...  
129              True  
130              True  
131              True  
132              True  
133              True  

[134 rows x 5 columns]

In [81]:
# Data cleaning
df['new_rank_score'] = df['new_rank_score'].fillna(0)
df.sort_values(by=['Country Name'], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country Name      134 non-null    object 
 1   Indicator         134 non-null    object 
 2   data_col          122 non-null    float64
 3   new_rank_score    134 non-null    float64
 4   higher_is_better  134 non-null    bool   
dtypes: bool(1), float64(2), object(2)
memory usage: 4.4+ KB


In [84]:
df.head(15)

Country Name                     Indicator  data_col  \
0                  Albania  Digital payments penetration      8.62   
1                  Algeria  Digital payments penetration      4.16   
2                   Angola  Digital payments penetration       NaN   
3                Argentina  Digital payments penetration     22.93   
4                  Armenia  Digital payments penetration     17.52   
5                Australia  Digital payments penetration     79.37   
6                  Austria  Digital payments penetration     59.89   
7               Azerbaijan  Digital payments penetration      6.93   
8                  Bahrain  Digital payments penetration     40.00   
9               Bangladesh  Digital payments penetration     14.08   
10                 Belarus  Digital payments penetration     46.02   
11                 Belgium  Digital payments penetration     73.15   
12                   Benin  Digital payments penetration     30.59   
13                 Bolivia  Digital payments penetration     12.41   
14  Bosnia and Herzegovina  Digital payments penetration     10.71   

    new_rank_score  higher_is_better  
0           1.4310              True  
1           1.2080              True  
2           0.0000              True  
3           2.1465              True  
4           1.8760              True  
5           4.9685              True  
6           3.9945              True  
7           1.3465              True  
8           3.0000              True  
9           1.7040              True  
10          3.3010              True  
11          4.6575              True  
12          2.5295              True  
13          1.6205              True  
14          1.5355              True

In [85]:
df.describe()

data_col  new_rank_score
count  122.000000      134.000000
mean    36.076967        2.552757
std     24.867406        1.432577
min      0.000000        0.000000
25%     16.235000        1.627375
50%     30.870000        2.400250
75%     51.110000        3.507125
max    100.000000        6.000000

In [86]:
# checking country names
sorted(df['Country Name'].unique().tolist())

['Albania',
 'Algeria',
 'Angola',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Benin',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo, Dem. Rep.',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 "Côte d'Ivoire",
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Finland',
 'France',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Guatemala',
 'Guinea',
 'Honduras',
 'Hong Kong (China)',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran, Islamic Rep.',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Korea, Rep.',
 'Kuwait',
 'Kyrgyzstan',
 'Lao PDR',
 'Latvia',
 'Lebanon',
 'Lesotho',
 'Lithuania',
 'Luxembourg',
 'Madagascar

In [87]:
# remove trailing whitespaces from country name
df['Country Name'] = df['Country Name'].str.strip()


In [88]:
# checking country names
sorted(df['Country Name'].unique().tolist())

['Albania',
 'Algeria',
 'Angola',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Benin',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo, Dem. Rep.',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 "Côte d'Ivoire",
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Finland',
 'France',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Guatemala',
 'Guinea',
 'Honduras',
 'Hong Kong (China)',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran, Islamic Rep.',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Korea, Rep.',
 'Kuwait',
 'Kyrgyzstan',
 'Lao PDR',
 'Latvia',
 'Lebanon',
 'Lesotho',
 'Lithuania',
 'Luxembourg',
 'Madagascar

In [89]:
# average indicator scores per country
agg_df = df.groupby(['Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [90]:
agg_df.columns = ['agg_score', 'count_source' ]

In [91]:
max_number_sources = agg_df.describe()['count_source']['max']

In [92]:
agg_df['agg_score_wt'] = agg_df['agg_score']*(agg_df['count_source']/max_number_sources)

In [93]:
agg_df.sort_values(by='agg_score', ascending=False, inplace=True)

In [94]:
agg_df.head(25)

agg_score  count_source  agg_score_wt
Country Name                                               
Norway                   6.0000             1        6.0000
Denmark                  5.8620             1        5.8620
Finland                  5.6975             1        5.6975
Sweden                   5.6540             1        5.6540
Netherlands              5.4505             1        5.4505
New Zealand              5.3365             1        5.3365
United States            5.2200             1        5.2200
Estonia                  5.1410             1        5.1410
Korea, Rep.              5.1320             1        5.1320
Canada                   5.0765             1        5.0765
Australia                4.9685             1        4.9685
Czech Republic           4.7825             1        4.7825
Belgium                  4.6575             1        4.6575
Germany                  4.5600             1        4.5600
Poland                   4.5170             1        4.5170
Latvia                   4.4680             1        4.4680
Luxembourg               4.3280             1        4.3280
Kenya                    4.3175             1        4.3175
Switzerland              4.3115             1        4.3115
Lithuania                4.0325             1        4.0325
France                   4.0310             1        4.0310
Austria                  3.9945             1        3.9945
United Arab Emirates     3.9580             1        3.9580
Slovakia                 3.9470             1        3.9470
Israel                   3.9245             1        3.9245

In [95]:
agg_df.to_csv('../pillar_scores/foundation_scores_v0.csv')